In [ ]:
from datetime import datetime, timedelta, date

hoje = date.today() + timedelta(days = 1)

final = hoje - timedelta(15)

hoje = hoje.strftime('%Y-%m-%d')
final = final.strftime('%Y-%m-%d')

dt_inicio = final
df_final = hoje

vlr_usado_teste = 3.01

dias = [ 5, 10, 15, 20, 30]

In [ ]:
#Tabela de feriados
import datetime

date_format = "%Y-%m-%d"

feriados = dict( [
        (datetime.datetime.strptime('2022-12-30', date_format), 'Ano Novo'), 
        (datetime.datetime.strptime('2022-12-26', date_format), 'Natal'),
        (datetime.datetime.strptime('2023-01-20', date_format), 'Carnaval'),
        (datetime.datetime.strptime('2023-01-21', date_format), 'Carnaval'),
        (datetime.datetime.strptime('2023-01-22', date_format), 'Carnaval'),
           
])

In [ ]:
df_out.createOrReplaceTempView("cte_pix_out_product")

In [ ]:
df_in.createOrReplaceTempView("cte_pix_in_product")

In [ ]:
df.createOrReplaceTempView("cte_pix_cc")

In [ ]:
df.createOrReplaceTempView("cte_pix_out")

In [ ]:
df.createOrReplaceTempView("cte_pix_in")

In [ ]:
SE_df = SE.toPandas()

In [ ]:
pix_raw = consulta_pix.toPandas()

In [ ]:
pix_in = df_in.toPandas()

In [ ]:
pix_out = df_out.toPandas()

In [ ]:
saldo_cc = saldo_cc.toPandas()

In [ ]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime
from functools import reduce


#cria os DataFrames
pix_df = pix_raw[['data_transacao', 'sum_pixout', 'sum_pixin', 'diff_sum']]
pix_in_df = pix_in[['data_transacao', 'amount']]
pix_out_df = pix_out [['data_transacao', 'amount']]
saldo_cc_df = saldo_cc[['data_lancamento', 'valor', 'descricao']]
pix_in_df_issuer = pix_in[['cliente_id', 'amount']]
pix_out_df_issuer = pix_out [['cliente_id', 'amount']]


#CRIA COLUNA DATA NOS DF IN E OUT
pix_in_df['date'] = pd.to_datetime(pix_in_df['data_transacao'].dt.date)
pix_out_df['date'] = pd.to_datetime(pix_out_df['data_transacao'].dt.date)

#ALTERA COLUNA 'data_lancamento' PARA FORMATO DATA
saldo_cc_df['data_lancamento'] = pd.to_datetime(saldo_cc_df['data_lancamento'])

# renomeia e completa com 0 (zero)
pix_df = pix_df.rename ( columns =  {'sum_pixout': 'pixout', 'sum_pixin':'pinxin', 'diff_sum':'Liq'} )

pix_df['dia_semana'] = pix_df['data_transacao'].dt.day_name()
pix_df['dia_semana_nr'] = pix_df['data_transacao'].dt.dayofweek
pix_df['dia'] = pix_df['data_transacao'].dt.day

pix_df['pixout'] = pix_df['pixout'].fillna(0)
pix_df['pinxin'] = pix_df['pinxin'].fillna(0)

#filtro de datas
pix_df_filtro = pix_df[ (pix_df['data_transacao'] >= dt_inicio) & (pix_df['data_transacao'] <= df_final)]
saldo_cc_df_filtro = saldo_cc_df[ (saldo_cc_df['data_lancamento'] >= dt_inicio) & (saldo_cc_df['data_lancamento'] <= df_final)]
pix_df_filtro_dia =  pix_df_filtro[ pix_df_filtro['dia'].isin(dias) ]



#FILTRA DATA E VALORES NO IN E OUT
pix_in_df_filtro = pix_in_df[ (pix_in_df['data_transacao'] >= dt_inicio) & (pix_in_df['data_transacao'] < df_final) & (pix_in_df['amount'] != vlr_usado_teste)]
pix_out_df_filtro = pix_out_df[ (pix_out_df['data_transacao'] >= dt_inicio) & (pix_out_df['data_transacao'] < df_final) & (pix_out_df['amount'] != vlr_usado_teste)]

#CRIA COLUNA COM A MEDIA MOVEL
pix_in_df_filtro['med_mov_20'] = pix_in_df['amount'].rolling(window=20, min_periods=0).mean()
pix_out_df_filtro['med_mov_20'] = pix_out_df['amount'].rolling(window=20, min_periods=0).mean()

#Calculos
SE_soma = pd.DataFrame(pix_df_filtro.groupby('data_transacao', as_index = False)['pinxin', 'pixout', 'Liq'].sum())
SE_soma.loc[:, 'pinxin'] = SE_soma['pinxin'].map('{:,.2f}'.format)
SE_soma.loc[:, 'pixout'] = SE_soma['pixout'].map('{:,.2f}'.format)
SE_soma.loc[:, 'Liq'] = SE_soma['Liq'].map('{:,.2f}'.format)
SE_soma['data_transacao'] = pd.to_datetime(SE_soma['data_transacao'],infer_datetime_format=False, dayfirst=True)

SE_soma_semana = pd.DataFrame(pix_df_filtro.groupby(['dia_semana_nr','dia_semana'], as_index = False)['pinxin', 'pixout', 'Liq'].sum()).drop(columns = 'dia_semana_nr')
SE_soma_semana.loc[:, 'pinxin'] = SE_soma_semana['pinxin'].map('{:,.2f}'.format)
SE_soma_semana.loc[:, 'pixout'] = SE_soma_semana['pixout'].map('{:,.2f}'.format)
SE_soma_semana.loc[:, 'Liq'] = SE_soma_semana['Liq'].map('{:,.2f}'.format)

SE_soma_dia = pd.DataFrame(pix_df_filtro_dia.groupby(['dia','dia_semana'], as_index = False)['pinxin', 'pixout', 'dia_semana'].sum())
SE_soma_dia.loc[:, 'pinxin'] = SE_soma_dia['pinxin'].map('{:,.2f}'.format)
SE_soma_dia.loc[:, 'pixout'] = SE_soma_dia['pixout'].map('{:,.2f}'.format)


pixin_maiores = pd.DataFrame(pix_in_df_filtro.sort_values(['amount'], ascending=False).head(10)).drop(columns = ['med_mov_20', 'date'])
pixin_maiores.loc[:, 'amount'] = pixin_maiores['amount'].map('{:,.2f}'.format)

pixout_maiores = pd.DataFrame(pix_out_df_filtro.sort_values(['amount'], ascending=False).head(10)).drop(columns = ['med_mov_20', 'date'])
pixout_maiores.loc[:, 'amount'] = pixout_maiores['amount'].map('{:,.2f}'.format)

pix_in_count = pix_in_df_filtro['amount'].count()
pix_out_count = pix_out_df_filtro['amount'].count()

pix_in_mean = pix_in_df_filtro['amount'].mean()
pix_out_mean = pix_out_df_filtro['amount'].mean()

pix_in_max =  pix_in_df_filtro['amount'].max()
pix_out_max = pix_out_df_filtro['amount'].max()

pix_in_min =  pix_in_df_filtro['amount'].min()
pix_out_min = pix_out_df_filtro['amount'].min()

pix_in_std =  pix_in_df_filtro['amount'].std()
pix_out_std = pix_out_df_filtro['amount'].std()

pix_in_median =  pix_in_df_filtro['amount'].median()
pix_out_median = pix_out_df_filtro['amount'].median()

q1_in = pix_in_df_filtro['amount'].quantile(.25)
q1_out = pix_out_df_filtro['amount'].quantile(.25)

q3_in = pix_in_df_filtro['amount'].quantile(.75)
q3_out = pix_out_df_filtro['amount'].quantile(.75)

iqr_in = q3_in - q1_in
iqr_out = q3_out - q1_out

Lsup_in = pix_in_mean + 1,5 * iqr_in
Lsup_out = pix_out_mean + 1,5 * iqr_out

Linf_in = pix_in_mean - 1,5 * iqr_in
Linf_out = pix_out_mean - 1,5 * iqr_out


#FERIADOS
pix_in_df_filtro_feriados = pd.DataFrame(pix_in_df_filtro[(pix_in_df_filtro['date'].isin(feriados) )] )
pix_out_df_filtro_feriados = pd.DataFrame(pix_out_df_filtro[(pix_out_df_filtro['date'].isin(feriados) )] )

pix_in_df_filtro_feriados['Feriado'] = pix_in_df_filtro_feriados['date'].map(feriados)
pix_out_df_filtro_feriados['Feriado'] = pix_out_df_filtro_feriados['date'].map(feriados)


pix_in_df_filtro_feriados = pd.DataFrame(pix_in_df_filtro_feriados.groupby(['Feriado', 'date'], as_index = False)['amount', 'Feriado', 'date'].sum())
pix_out_df_filtro_feriados = pd.DataFrame(pix_out_df_filtro_feriados.groupby(['Feriado', 'date'], as_index = False)['amount', 'Feriado', 'date'].sum())

pix_in_df_filtro_feriados.loc[:, 'amount'] = pix_in_df_filtro_feriados['amount'].map('{:,.2f}'.format)
pix_out_df_filtro_feriados.loc[:, 'amount'] = pix_out_df_filtro_feriados['amount'].map('{:,.2f}'.format)


saldo_cc_df_filtro.loc[:, 'valor'] = saldo_cc_df_filtro['valor'].map('{:,.2f}'.format)



In [ ]:
# agrupa por faixa de valor

df_in = pix_in_df_filtro

faixas_in = np.arange(0.01, pix_in_df_filtro['amount'].max()+1, 1000)

df_in['faixa'] = pd.cut(df_in['amount'], faixas_in)

df_agrupado_in = pd.DataFrame(df_in.groupby('faixa')['amount'].count())
df_agrupado_in = df_agrupado_in[df_agrupado_in['amount'] >= 50]

total_grupo_in = df_agrupado_in['amount'].sum()
per_grupo_in = (total_grupo_in / pix_in_count) * 100 

df_agrupado_in.loc[:, 'amount'] = df_agrupado_in['amount'].map('{:,.0f}'.format)
df_agrupado_in = df_agrupado_in.rename(columns = {'amount':'quanidade' } )

######


df_out = pix_out_df_filtro

faixas_out = np.arange(0.01, pix_out_df_filtro['amount'].max()+1, 1000)

df_out['faixa'] = pd.cut(df_out['amount'], faixas_out)

df_agrupado_out = pd.DataFrame(df_out.groupby('faixa')['amount'].count())
df_agrupado_out = df_agrupado_out[df_agrupado_out['amount'] >= 50]

total_grupo_out = df_agrupado_out['amount'].sum()
per_grupo_out = ( total_grupo_out / pix_out_count ) * 100

df_agrupado_out.loc[:, 'amount'] = df_agrupado_out['amount'].map('{:,.0f}'.format)
df_agrupado_out = df_agrupado_out.rename(columns = {'amount':'quanidade' } )



In [ ]:
# valor por SE

pin_issuer = pix_in_df_issuer.groupby('cliente_id', as_index = False)['amount'].sum()
pin_issuer.rename ( columns =  {'amount': 'pixin'}, inplace = True )

pout_issuer = pix_out_df_issuer.groupby('cliente_id', as_index = False)['amount'].sum()
pout_issuer.rename ( columns =  {'amount': 'pixout'}, inplace = True )


issuer_df = pin_issuer.merge ( pout_issuer, how='left', on='cliente_id' )
issuer_df['pixout'].fillna(0, inplace=True)
issuer_df['pixin'].fillna(0, inplace=True)

issuer_df['Liq'] = issuer_df['pixin'] - issuer_df['pixout']

issuer_df.sort_values(['Liq'], ascending=True, inplace = True )

issuer_df.loc[:, 'pixout'] = issuer_df['pixout'].map('{:,.2f}'.format)
issuer_df.loc[:, 'pixin'] = issuer_df['pixin'].map('{:,.2f}'.format)
issuer_df.loc[:, 'Liq'] = issuer_df['Liq'].map('{:,.2f}'.format)

issuer_df = issuer_df.merge ( SE_df, how='inner', on='cliente_id' )


In [ ]:
# Relatório
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)


titulo = 'Pix Externo - liquidado pelo 301'
fim = 'FIM DO RELATÓRIO'
volume = '---- VOLUMES ----'
top10 = '--- TOP 10 ---'
est = '--- DADOS ESTATISTICOS ---'
feriados = '---- FERIADOS ----'
dia = f'---- DIAS SELECIONADOS: {dias} ----'
qtd = f'--- QUANTIDADE DE OPERACOES POR FAIXA DE VALOR ---'
vlr = f'---- Valor por SE ----'

print('*' * 67)
print(titulo.center(67))
print('*' * 67)

print('Período de análise: ', dt_inicio, ' até ', df_final)
print(f'Total de transações no período: Pix-in: {pix_in_count:,}| Pix-out:{pix_out_count:,}')
print('\n')

print(volume.center(67))
print('Valor total por dia')
sparkDF1=spark.createDataFrame(SE_soma)
sparkDF1.show(500,truncate = False)
print('Valor total por dia - semana')
sparkDF11=spark.createDataFrame(SE_soma_semana)
sparkDF11.show(truncate = False)


print(top10.center(67))
print('*| Pix - in |*')
sparkDF2=spark.createDataFrame(pixin_maiores)
sparkDF2.show(500,truncate = False)
print('*| Pix - out |*')
sparkDF3=spark.createDataFrame(pixout_maiores)
sparkDF3.show(500,truncate = False)

print(est.center(67))
print('*| Média |*')
print(f'Pix-in: {pix_in_mean:,.2f}')
print(f'Pix-Out: {pix_out_mean:,.2f}')
print('\n')
print('*| Pix - in |*')
print(f'Min: {pix_in_min:,.2f}')
print(f'Q1(25): {q1_in:,.2f}')
print(f'Mediana: {pix_in_median} ')
print(f'Q3(75): {q3_in:,.2f}')
print(f'Max: {pix_in_max:,.2f} ')
print('Outlier:')
print(f'Limite superior: {Lsup_in[0]:,.2f}')
print(f'Limite inferior: {Linf_in[0]:,.2f}')
print('\n')

print('*| Pix - out |*')
print(f'Min: {pix_out_min:,.2f}')
print(f'Q1(25): {q1_out:,.2f}')
print(f'Mediana: {pix_out_median} ')
print(f'Q3(75): {q3_out:,.2f}')
print(f'Max: {pix_out_max:,.2f} ')
print('Outlier:')
print(f'Limite superior: {Lsup_out[0]:,.2f}')
print(f'Limite inferior: {Linf_out[0]:,.2f}')
print('\n')
print(feriados.center(67))
print('*| Pix - in |*')
if pix_in_df_filtro_feriados.empty == True:
    print('')
else:
    sparkDF12=spark.createDataFrame(pix_in_df_filtro_feriados)
    sparkDF12.show(truncate = False)
    print(sparkDF12)

print('\n')
print('*| Pix - out |*')
if pix_out_df_filtro_feriados.empty == True:
    print('')
else:
    sparkDF13=spark.createDataFrame(pix_out_df_filtro_feriados)
    sparkDF13.show(truncate = False)
    print(sparkDF13)

print('\n')
print(dia.center(67))
sparkDF14=spark.createDataFrame(SE_soma_dia)
sparkDF14.show(truncate = False)
print(sparkDF14)
print('\n')

print(qtd.center(67))
print('faixas com quantidades maiores que 50')
print('\n')
print('*| Pix - IN |*')
print(df_agrupado_in)
print(f'Total: {total_grupo_in:,.0f} representa {per_grupo_in:,.4f}% do total de operacoes')
print('\n')

print('*| Pix - out |*')
print(df_agrupado_out)
print(f'Total: {total_grupo_out:,.0f} representa {per_grupo_out:,.4f}% do total de operacoes')
print('\n')

print(vlr.center(67))
print('\n')
print(issuer_df.set_index('cliente_id'))
      
print('\n')      
print('---- Saldos: 2470674 ----')
print(saldo_cc_df_filtro.set_index('data_lancamento').head(10))
print('\n')
print('\n')

print('Emitido em: ',datetime.now().strftime('%d/%m/%Y %H:%M:%S'))
print('*' * 67)
print(fim.center(67))
print('*' * 67)

*******************************************************************
                  Pix Externo - liquidado pelo banco                 
*******************************************************************
Período de análise:  2023-01-26  até  2023-02-10
Total de transações no período: Pix-in: 0000| Pix-out: 00000


                         ---- VOLUMES ----                         
Valor total por dia
+-------------------+-------------+--------------+--------------+
|transaction_date   |pinxin       |pixout        |Liq           |
+-------------------+-------------+--------------+--------------+
|2023-01-26 00:00:00|             |              |              |
|2023-01-27 00:00:00|             |              |              |
|2023-01-28 00:00:00|             |              |              |
|2023-01-29 00:00:00|             |              |              |
|2023-01-30 00:00:00|             |              |              |
|2023-01-31 00:00:00|             |              |              |
|2023-02-01 00:00:00|             |              |              |
|2023-02-02 00:00:00|             |              |              |
|2023-02-03 00:00:00|             |              |              |
|2023-02-04 00:00:00|             |              |              |
|2023-02-05 00:00:00|             |              |              |
|2023-02-06 00:00:00|             |              |              |
|2023-02-07 00:00:00|             |              |              |
|2023-02-08 00:00:00|             |              |              |
|2023-02-09 00:00:00|             |              |              |
+-------------------+-------------+--------------+--------------+

Valor total por dia - semana
+----------+-------------+--------------+-------------+
|dia_semana|pinxin       |pixout        |Liq          |
+----------+-------------+--------------+-------------+
|Monday    |             |              |             |
|Tuesday   |             |              |             |
|Wednesday |             |              |             |
|Thursday  |             |              |             |
|Friday    |             |              |             |
|Saturday  |             |              |             |
|Sunday    |             |              |             |
+----------+-------------+--------------+-------------+

                           --- TOP 10 ---                          
*| Pix - in |*
+-----------------------+------------+
|transaction_date       |amount      |
+-----------------------+------------+
|2023-01-27 hh:mm:ss    |            |
|2023-01-27             |            |
|2023-01-26             |            |
|2023-01-26             |            |
|2023-02-03             |            |
|2023-02-03             |            |
|2023-02-03             |            |
|2023-02-01             |            |
|2023-01-27             |            |
|2023-02-03             |            |
+-----------------------+------------+

*| Pix - out |*
+------------------------+------------+
|transaction_date        |amount      |
+------------------------+------------+
|2023-01-26 hh:mm:ss     |            |
|2023-02-03              |            |
|2023-02-03              |            |
|2023-02-07              |            |
|2023-02-03              |            |
|2023-01-27              |            |
|2023-02-02              |            |
|2023-02-08              |            |
|2023-02-02              |            |
|2023-02-01              |            |
+------------------------+------------+

                     --- DADOS ESTATISTICOS ---                    
*| Média |*
Pix-in: 0000000
Pix-Out: 00000000


*| Pix - in |*
Min: 00000
Q1(25): 0000
Mediana: 0000 
Q3(75): 000000
Max: 0000000 
Outlier:
Limite superior: 00000
Limite inferior: 00000


*| Pix - out |*
Min: 000000
Q1(25): 000000
Mediana: 00000 
Q3(75): 0000000
Max: 0000000 
Outlier:
Limite superior: 0000
Limite inferior: 00000


                         ---- FERIADOS ----                        
*| Pix - in |*



*| Pix - out |*



          ---- DIAS SELECIONADOS: [5, 10, 15, 20, 30] ----         
+---+----------+-------------+--------------+
|dia|dia_semana|pinxin       |pixout        |
+---+----------+-------------+--------------+
|5  |Sunday    |             |              |
|30 |Monday    |             |              |
+---+----------+-------------+--------------+

DataFrame[dia: bigint, dia_semana: string, pinxin: string, pixout: string]


         --- QUANTIDADE DE OPERACOES POR FAIXA DE VALOR ---        
faixas com quantidades maiores que 50


*| Pix - IN |*
                      quanidade
faixa                          
(0.01, 1000.01]         
(1000.01, 2000.01]      
(2000.01, 3000.01]      
(3000.01, 4000.01]      
(4000.01, 5000.01]      
(5000.01, 6000.01]      
(6000.01, 7000.01]      
(7000.01, 8000.01]      
(8000.01, 9000.01]      
(9000.01, 10000.01]     
(10000.01, 11000.01]    
(11000.01, 12000.01]    
(12000.01, 13000.01]    
(13000.01, 14000.01]    
(14000.01, 15000.01]    
(15000.01, 16000.01]    
(16000.01, 17000.01]    
(17000.01, 18000.01]    
(18000.01, 19000.01]    
(19000.01, 20000.01]    
(20000.01, 21000.01]    
(21000.01, 22000.01]    
(22000.01, 23000.01]    
(23000.01, 24000.01]    
(24000.01, 25000.01]    
(25000.01, 26000.01]    
(26000.01, 27000.01]    
(27000.01, 28000.01]    
(29000.01, 30000.01]    
(34000.01, 35000.01]    
(39000.01, 40000.01]    
(49000.01, 50000.01]    
(59000.01, 60000.01]    
(99000.01, 100000.01]   
Total: 00000000 representa 99.6208% do total de operacoes


*| Pix - out |*
                     quanidade
faixa                         
(0.01, 1000.01]        
(1000.01, 2000.01]     
(2000.01, 3000.01]     
(3000.01, 4000.01]     
(4000.01, 5000.01]     
(5000.01, 6000.01]     
(6000.01, 7000.01]     
(7000.01, 8000.01]     
(8000.01, 9000.01]     
(9000.01, 10000.01]    
(10000.01, 11000.01]   
(11000.01, 12000.01]   
(12000.01, 13000.01]   
(13000.01, 14000.01]   
(14000.01, 15000.01]   
(15000.01, 16000.01]   
(16000.01, 17000.01]   
(17000.01, 18000.01]   
(18000.01, 19000.01]   
(19000.01, 20000.01]   
(20000.01, 21000.01]   
(21000.01, 22000.01]   
(22000.01, 23000.01]   
(23000.01, 24000.01]   
(24000.01, 25000.01]   
(29000.01, 30000.01]   
(39000.01, 40000.01]   
(49000.01, 50000.01]   
Total: 0000000 representa 99.7425% do total de operacoes


                       ---- Valor por SE ----                      


                    pixin          pixout             Liq  cliente
cliente_id                                                             



---- Saldos: 00000000 ----
                         valor  descricao
data_lancamento                          
2023-02-07                       SALDO C/C
2023-02-06                       SALDO C/C
2023-02-03                       SALDO C/C
2023-02-02                       SALDO C/C
2023-02-01                       SALDO C/C
2023-01-31                       SALDO C/C
2023-01-30                       SALDO C/C
2023-01-27                       SALDO C/C
2023-01-26                       SALDO C/C




Emitido em:  09/02/2023 13:46:09
*******************************************************************
                          FIM DO RELATÓRIO                         
*******************************************************************